# Street Fighter PPO
This notebook shows how to create the environment without the delta transformation. 

In [ ]:
import retro

In [ ]:
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

In [ ]:
obs = env.reset()

In [ ]:
obs = env.reset()
done = False
for game in range(5):
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        print(reward)

In [ ]:
info

In [ ]:
env.action_space.sample()

In [ ]:
env.observation_space.sample()

In [ ]:
env.close()

In [ ]:
from gym import Env
from gym.spaces import Box, MultiBinary
import numpy as np
import cv2
import time

In [ ]:
class StreetFighter(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
        #self.score = 0
    
    def step(self, action):
        lst=[0]
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)
        
       
        frame_delta = obs 

        ##-- Shape reward--##
        
        #1st Variation 
        
        reward = info['score'] - self.score 
        self.score = info['score']
        
        
        
        #2nd Variation ((+) for doing damage or winning,(-) for taking damage or losing based on health and enemy_health)
        
        #if  info['health']==0 and info['enemy_health']==0:
         #   reward=0
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health']
            
        #if  info['health']<0 and info['health']!=self.health and info['enemy_health']!=0:
         #   reward=(-176+((info['health'] - self.health)))*info['enemy_health']
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health']
            
        #elif info['enemy_health']<0 and info['enemy_health']!=self.enemy_health and info['health']!=0:
         #   reward=(176-(info['enemy_health'] - self.enemy_health))*(info['health'])
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health'] 
            
        #else: 
         #   reward=((info['health'] - self.health))-(info['enemy_health'] - self.enemy_health)
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health']
     
    
            
         #3rd Variation 
        
        #rew2=((-176+(info['health']-self.health))* info['enemy_health'])
        #rew4=(info['health'] - self.health)*10 #dmg absorbed
        
        
       # if  info['health']==0 and info['enemy_health']==0:
        #    reward=0
         #   self.enemy_health = info['enemy_health']
          #  self.health = info['health']
            
        #elif info['health']<0 and info['health']!=self.health and info['enemy_health']!=0:
         #   reward=rew2+(rew2*info['enemy_matches_won']/10)  #καποια σταθερα
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health']
            #self.score = info['score']
            
        #elif info['enemy_health']<0 and info['enemy_health']!=self.enemy_health and info['health']!=0:
         #   reward = info['score'] - self.score 
          #  self.score = info['score']
           # self.enemy_health = info['enemy_health']
            #self.health = info['health']
            
        #else:
         #   if  (info['health']< self.health) and (info['enemy_health']==self.enemy_health): 
          #      reward= rew4+(rew4*info['enemy_matches_won']/10)
           #     self.enemy_health = info['enemy_health']
            #    self.health = info['health']
             #   self.score = info['score']
                
            #elif (info['health']< self.health) and (info['enemy_health']< self.enemy_health):
             #    reward= 10*(((info['health'] - self.health))-(info['enemy_health'] - self.enemy_health))#den pairnei score se isopalia
              #   self.enemy_health = info['enemy_health']
               #  self.health = info['health']   
                # print('double_damage',reward)
            #else:
             #    reward = (info['score'] - self.score)   
              #   self.score = info['score']
               #  self.enemy_health = info['enemy_health']
                # self.health = info['health']
                                                        
        return frame_delta, reward, done, info, 
    
    
    def render(self, *args, **kwargs): 
        self.game.render()
    
    def reset(self):
        self.previous_frame = np.zeros(self.game.observation_space.shape)
        
        # Frame delta
        obs = self.game.reset()
        obs = self.preprocess(obs)
        self.previous_frame = obs
        
        # Create initial variables
        self.score = 0       
        #self.enemy_health=0
        #self.health=0
    
        return obs
    
    def preprocess(self, observation): 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (84,84,1))
        return state
    
    def close(self): 
        self.game.close()

In [ ]:
env=StreetFighter()

In [ ]:
import time

In [ ]:
obs = env.reset()
done = False
for game in range(5):
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        if reward!=0:
           print(reward,info['health'],info['enemy_health'], info['score'], info['matches_won'], info['enemy_matches_won'])
        time.sleep(0.01)

In [ ]:
# Import PPO 
from stable_baselines3 import PPO
# Evaluate Policy
from stable_baselines3.common.evaluation import evaluate_policy
# Import wrappers
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import os 

In [ ]:
LOG_DIR = './logs/'
OPT_DIR = './opt_nodelta/'

In [ ]:
import os 
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train_ppo_def_ver2/'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
env.close()

In [ ]:
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
model_params = {'n_steps': 2048, 'gamma': 0.99, 'learning_rate': 0.0003, 'clip_range': 0.2, 'gae_lambda': 0.95, 'ent_coef': 0.0, 'vf_coef': 0.5, 'max_grad_norm': 0.5}# def

In [ ]:
#For solving memory type of errors
import torch
import gc

gc.collect()

torch.cuda.empty_cache()
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)


In [ ]:
model.learn(total_timesteps=10000000, callback=callback)

In [ ]:
env.close()

In [ ]:
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
final_reward=0
won_en=[]
for episode in range(10): 
    obs = env.reset()
    done = False
    total_reward = 0
    enemies_won=0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        env.render()
        #time.sleep(0.001)   
        total_reward += reward
        if info['matches_won']==2:              #οτι νικησε δλδ εναν αντιπαλο
            enemies_won+=1
    won_en.append(enemies_won)

    print('Total Reward for episode {} is {} and total enemies won is {}'.format(episode, total_reward, enemies_won))   
    final_reward+=total_reward   
print('Final Reward for 10 episodes is', final_reward, won_en)
